In [ ]:
# -----------------------------------------------------------
# 🧩 Step 1: Chip TIFFs and Rasterize Masks
#
# This step creates 1600x1600 image/mask tiles from each map set
# - Uses 10% overlap (stride = 1440)
# - Includes background-only tiles
# - Rasterizes shapefile features to create mask chips
# - Outputs tile metadata for later preview and reconstruction
#
# Outputs:
# - /[Map Folder]/tiled/images/
# - /[Map Folder]/tiled/masks/
# - /[Map Folder]/tiled/tile_metadata.csv
# - /[Map Folder]/tiled/raster_shape.txt
# -----------------------------------------------------------

import os
import csv
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
import cv2
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"
chip_size = 1600
stride = 1440  # 10% overlap

all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]

# --- Main loop over all map sets ---
for map_folder in all_maps:
    print(f"\n🧩 Processing: {map_folder}")
    map_base = os.path.join(base_dir, map_folder)
    tif_path = os.path.join(map_base, f"{map_folder}.tiff")
    shp_path = os.path.join(map_base, f"{map_folder}.shp")

    out_img_dir = os.path.join(map_base, "tiled", "images")
    out_mask_dir = os.path.join(map_base, "tiled", "masks")
    meta_csv = os.path.join(map_base, "tiled", "tile_metadata.csv")
    shape_txt = os.path.join(map_base, "tiled", "raster_shape.txt")

    # Create fresh output folders
    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_mask_dir, exist_ok=True)

    with rasterio.open(tif_path) as raster:
        labels = gpd.read_file(shp_path)
        if labels.crs != raster.crs:
            labels = labels.to_crs(raster.crs)

        # Save raster shape for stitching
        with open(shape_txt, "w") as f:
            f.write(f"{raster.height},{raster.width}")

        with open(meta_csv, mode="w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["filename", "x", "y", "width", "height"])  # Updated header

            count = 0
            for y in tqdm(range(0, raster.height - chip_size + 1, stride), desc=f"Tiling {map_folder}"):
                for x in range(0, raster.width - chip_size + 1, stride):
                    window = Window(x, y, chip_size, chip_size)
                    transform = raster.window_transform(window)
                    bounds = box(*rasterio.windows.bounds(window, raster.transform))

                    intersecting = labels[labels.intersects(bounds)]

                    # --- Save image tile ---
                    image = raster.read([1, 2, 3], window=window)
                    image = np.transpose(image, (1, 2, 0))
                    img_filename = f"chip_{count}.png"
                    cv2.imwrite(os.path.join(out_img_dir, img_filename), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

                    # --- Save mask (empty if no features) ---
                    if not intersecting.empty:
                        shapes = [(geom, cid) for geom, cid in zip(intersecting.geometry, intersecting["class_id"])]
                        mask = rasterize(
                            shapes,
                            out_shape=(chip_size, chip_size),
                            transform=transform,
                            fill=0,
                            dtype=np.uint8
                        )
                    else:
                        mask = np.zeros((chip_size, chip_size), dtype=np.uint8)

                    cv2.imwrite(os.path.join(out_mask_dir, img_filename), mask)

                    # --- Record metadata ---
                    writer.writerow([img_filename, x, y, chip_size, chip_size])
                    count += 1

    print(f"✅ Created {count} tile+mask pairs for {map_folder}")

In [1]:
# -----------------------------------------------------------
# ⚡ Step 2: Convert Mask Tiles to YOLOv8 Polygon Labels
#
# This step converts rasterized mask tiles into YOLOv8-style
# polygon .txt labels, enabling training with segmentation models.
#
# For each map folder:
# - Loads corresponding image and mask tiles
# - Resizes both to 640×640 (matching YOLOv8 input size)
# - Extracts external contours for each class in the mask
# - Writes polygons in YOLO format:
#     • class_id x1 y1 x2 y2 ... xn yn (normalized coordinates)
#
# Enhancements:
# - Runs in parallel using multithreading for speed
# - Background-only tiles are preserved with empty label files
# - Automatically splits 70/30 into train and val subsets
#
# Input:
# - /[map]/tiled/images/      ← RGB tiles
# - /[map]/tiled/masks/       ← Grayscale masks (class IDs)
#
# Output:
# - /[map]/yolo_dataset_640/images/train/val/
# - /[map]/yolo_dataset_640/labels/train/val/
#
# Notes:
# - Supports 3-class masks: 0 = Water, 1 = Road, 2 = PVeg
# - No remapping is needed if classes are already 0-based
# - This format is compatible with YOLOv8-seg training
# -----------------------------------------------------------

import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Configuration ---
base_dir = "C:/QGIS"
target_size = 1024
num_threads = 8

all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek",
    "Project_2024_09_20"
]

def mask_to_polygons(mask):
    contours = {}
    for cls_id in np.unique(mask):
        if cls_id == 0:
            continue
        binary = (mask == cls_id).astype(np.uint8)
        cnts, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            contours[cls_id] = cnts
    return contours

def process_file(fname, img_input_dir, mask_input_dir, img_out, lbl_out):
    img_path = os.path.join(img_input_dir, fname)
    mask_path = os.path.join(mask_input_dir, fname)

    try:
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or mask is None:
            return False  # Skip bad data

        img_resized = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)
        mask_resized = cv2.resize(mask, (target_size, target_size), interpolation=cv2.INTER_NEAREST)
        cv2.imwrite(os.path.join(img_out, fname), img_resized)

        label_path = os.path.join(lbl_out, fname.replace(".png", ".txt"))
        contours = mask_to_polygons(mask_resized)

        if not contours:
            open(label_path, "w").close()
            return True  # Background-only tile

        with open(label_path, "w") as f:
            for cls_id, cnts in contours.items():
                for cnt in cnts:
                    if len(cnt) < 3:
                        continue
                    pts = cnt.reshape(-1, 2).astype(np.float32) / target_size
                    coords = " ".join(f"{x:.6f} {y:.6f}" for x, y in pts)
                    f.write(f"{cls_id} {coords}\n")

        return False
    except Exception as e:
        print(f"⚠️ Error processing {fname}: {e}")
        return False

# --- Process each map ---
for map_folder in all_maps:
    print(f"\n🧩 Converting: {map_folder}")

    base_map_dir = os.path.join(base_dir, map_folder)
    img_input_dir = os.path.join(base_map_dir, "tiled", "images")
    mask_input_dir = os.path.join(base_map_dir, "tiled", "masks")
    out_base = os.path.join(base_map_dir, "yolo_dataset_640")

    out_dirs = {
        "train_img": os.path.join(out_base, "images", "train"),
        "val_img": os.path.join(out_base, "images", "val"),
        "train_lbl": os.path.join(out_base, "labels", "train"),
        "val_lbl": os.path.join(out_base, "labels", "val"),
    }
    for d in out_dirs.values():
        os.makedirs(d, exist_ok=True)

    chip_files = [f for f in os.listdir(img_input_dir) if f.endswith(".png")]
    random.shuffle(chip_files)
    split_idx = int(len(chip_files) * 0.7)
    train_files = chip_files[:split_idx]
    val_files = chip_files[split_idx:]

    for mode, files in [("train", train_files), ("val", val_files)]:
        img_out = out_dirs[f"{mode}_img"]
        lbl_out = out_dirs[f"{mode}_lbl"]
        bg_count = 0

        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(process_file, f, img_input_dir, mask_input_dir, img_out, lbl_out) for f in files]
            for f in tqdm(as_completed(futures), total=len(futures), desc=f"{map_folder} [{mode}]"):
                if f.result():
                    bg_count += 1

        print(f"✅ {map_folder} [{mode}]: {len(files)} tiles processed ({bg_count} background-only)")


🧩 Converting: Bear_Creek_20250112


Bear_Creek_20250112 [train]: 100%|█████████████████████████████████████████████████| 2049/2049 [01:36<00:00, 21.23it/s]


✅ Bear_Creek_20250112 [train]: 2049 tiles processed (1328 background-only)


Bear_Creek_20250112 [val]: 100%|█████████████████████████████████████████████████████| 879/879 [00:42<00:00, 20.83it/s]


✅ Bear_Creek_20250112 [val]: 879 tiles processed (598 background-only)

🧩 Converting: Bear_Lane


...

✅ Wildcat_Creek [val]: 452 tiles processed (332 background-only)

🧩 Converting: Project_2024_09_20


Project_2024_09_20 [train]: 100%|██████████████████████████████████████████████████| 1337/1337 [01:06<00:00, 20.21it/s]


✅ Project_2024_09_20 [train]: 1337 tiles processed (831 background-only)


Project_2024_09_20 [val]: 100%|██████████████████████████████████████████████████████| 574/574 [00:30<00:00, 19.08it/s]

✅ Project_2024_09_20 [val]: 574 tiles processed (322 background-only)


In [2]:
# -----------------------------------------------------------
# 📦 Unified YOLO Dataset Builder (Excludes Holdout Set)
#
# This script merges YOLO-formatted image and label tiles from
# multiple map sets into a single unified dataset structure.
#
# Excludes holdout map: Flight_2_25pct
# Output folders:
# - dataset/images/train/
# - dataset/images/val/
# - dataset/labels/train/
# - dataset/labels/val/
# - dataset/manifests/train_files.txt and val_files.txt
# -----------------------------------------------------------

import os
import shutil
import random
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"
dataset_dir = os.path.join(base_dir, "dataset")
img_train_dir = os.path.join(dataset_dir, "images", "train")
img_val_dir = os.path.join(dataset_dir, "images", "val")
lbl_train_dir = os.path.join(dataset_dir, "labels", "train")
lbl_val_dir = os.path.join(dataset_dir, "labels", "val")
manifest_dir = os.path.join(dataset_dir, "manifests")
train_split = 0.8

holdout_set = "Flight_2_25pct"
all_sets = [
    "Bear_Creek_20250112", "Bear_Lane", "Flight_2",
    "SFLBC", "Sugar_Refugia_20241112", "Wildcat_Creek", "Project_2024_09_20"
]  # Excludes holdout

for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir, manifest_dir]:
    os.makedirs(d, exist_ok=True)

train_list, val_list = [], []

# --- Merge sets ---
for set_name in tqdm(all_sets, desc="Merging sets"):
    base_map_dir = os.path.join(base_dir, set_name)
    img_base = os.path.join(base_map_dir, "yolo_dataset_640", "images")
    lbl_base = os.path.join(base_map_dir, "yolo_dataset_640", "labels")

    # Collect all labeled image paths
    all_images = []
    for split in ["train", "val"]:
        img_dir = os.path.join(img_base, split)
        lbl_dir = os.path.join(lbl_base, split)
        if os.path.exists(img_dir):
            for fname in os.listdir(img_dir):
                if fname.endswith(".png"):
                    all_images.append((os.path.join(img_dir, fname),
                                       os.path.join(lbl_dir, fname.replace(".png", ".txt"))))

    random.shuffle(all_images)
    split_idx = int(len(all_images) * train_split)
    split_files = [("train", all_images[:split_idx]), ("val", all_images[split_idx:])]

    for mode, files in split_files:
        img_out_dir = img_train_dir if mode == "train" else img_val_dir
        lbl_out_dir = lbl_train_dir if mode == "train" else lbl_val_dir
        manifest = train_list if mode == "train" else val_list

        for img_path, lbl_path in files:
            new_img_name = f"{set_name}_{os.path.basename(img_path)}"
            new_lbl_name = new_img_name.replace(".png", ".txt")

            shutil.copy(img_path, os.path.join(img_out_dir, new_img_name))
            if os.path.exists(lbl_path):
                shutil.copy(lbl_path, os.path.join(lbl_out_dir, new_lbl_name))
            else:
                open(os.path.join(lbl_out_dir, new_lbl_name), "w").close()

            manifest.append(f"images/{mode}/{new_img_name}")

# --- Save manifest files ---
with open(os.path.join(manifest_dir, "train_files.txt"), "w") as f:
    f.write("\n".join(sorted(train_list)) + "\n")

with open(os.path.join(manifest_dir, "val_files.txt"), "w") as f:
    f.write("\n".join(sorted(val_list)) + "\n")

print(f"✅ Merged {len(train_list)} train / {len(val_list)} val tiles into: {dataset_dir}")
print(f"ℹ️ Held-out map: {holdout_set}")

Merging sets: 100%|██████████████████████████████████████████████████████████████████████| 7/7 [03:58<00:00, 34.07s/it]

✅ Merged 8922 train / 2234 val tiles into: C:/QGIS\dataset
ℹ️ Held-out map: Flight_2_25pct


In [ ]:
# --- Training done in Train.py 
import os
from ultralytics import YOLO
import torch

def main():
    torch.cuda.empty_cache()

    dataset_dir = "C:/QGIS/dataset"
    model_type = "yolov8s-seg.pt"
    img_size = 1024
    save_dir = "C:/QGIS/runs/segment/train"
    class_names = ["Water", "Road", "PVeg"]

    data_yaml = os.path.join(dataset_dir, "data.yaml")
    with open(data_yaml, "w") as f:
        f.write(f"path: {dataset_dir}\n")
        f.write("train: images/train\n")
        f.write("val: images/val\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write(f"names: {class_names}\n")

    print(f"✅ data.yaml written to: {data_yaml}")

    model = YOLO(model_type)
    model.train(
        data=data_yaml,
        imgsz=img_size,
        epochs=50,
        batch=16,
        workers=2,
        amp=True,
        patience=5,
        device="cuda",
        save=True,
        save_period=-1,
        project=save_dir,
        name="main_model",
        verbose=True,
        plots=False,
        cache=True
    )

if __name__ == "__main__":
    main()


In [ ]:
# Model Report

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# --- Paths ---
results_path = "C:/QGIS/runs/segment/train12/results.csv"
report_path = "C:/QGIS/diagnostics/model_report.pdf"
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# --- Load results and clean columns ---
df = pd.read_csv(results_path)
df.columns = df.columns.str.strip()

# --- Key epoch metrics ---
final_epoch = df.iloc[-1]
best_map_epoch = df["metrics/mAP50(B)"].idxmax()
best_row = df.iloc[best_map_epoch]

# --- Basic training config (update if needed) ---
training_config = {
    "Model": "YOLOv8n-seg",
    "Input Size": "640x640",
    "Epochs": len(df),
    "Batch Size": 8,
    "Optimizer": "SGD (default)",
    "Confidence Threshold": 0.1,
    "Tile Size": "1024x1024",
    "Mask Source": "Polygon label → Raster mask via cv2.fillPoly"
}

# --- Metrics to plot ---
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5–0.95",
    "metrics/precision(B)": "Precision",
    "metrics/recall(B)": "Recall"
}
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

# --- Create PDF report ---
with PdfPages(report_path) as pdf:
    # Title + Summary page
    plt.figure(figsize=(11, 8.5))
    plt.text(0.5, 0.78, "Model Performance Report", ha="center", fontsize=24)
    plt.text(0.5, 0.70, "Project: River and Road Segmentation", ha="center", fontsize=14)
    plt.text(0.5, 0.63, f"Best mAP@0.5: {best_row['metrics/mAP50(B)']:.3f} at epoch {best_map_epoch}", ha="center", fontsize=12)
    plt.text(0.5, 0.57, f"Final Epoch Precision: {final_epoch['metrics/precision(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.51, f"Final Epoch Recall: {final_epoch['metrics/recall(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.45, "Report generated from YOLOv8 results.csv", ha="center", fontsize=10)
    plt.axis("off")
    pdf.savefig()
    plt.close()

    # Training configuration page
    fig, ax = plt.subplots(figsize=(11, 4))
    ax.axis("off")
    table_data = list(training_config.items())
    table = ax.table(cellText=table_data, colLabels=["Parameter", "Value"], loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.4)
    plt.title("Training Configuration", fontsize=14)
    pdf.savefig()
    plt.close()

    # Metric plots
    for i, (key, label) in enumerate(metrics.items()):
        if key in df.columns:
            plt.figure(figsize=(10, 4))
            plt.plot(df[key], marker='o', linewidth=2, color=colors[i])
            plt.title(f"{label} Over Epochs")
            plt.xlabel("Epoch")
            plt.ylabel(label)
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    # Performance summary table
    summary_data = {
        "Metric": ["mAP@0.5", "mAP@0.5–0.95", "Precision", "Recall"],
        "Final Epoch": [
            f"{final_epoch.get('metrics/mAP50(B)', 0):.3f}",
            f"{final_epoch.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{final_epoch.get('metrics/precision(B)', 0):.3f}",
            f"{final_epoch.get('metrics/recall(B)', 0):.3f}"
        ],
        f"Best Epoch ({best_map_epoch})": [
            f"{best_row.get('metrics/mAP50(B)', 0):.3f}",
            f"{best_row.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{best_row.get('metrics/precision(B)', 0):.3f}",
            f"{best_row.get('metrics/recall(B)', 0):.3f}"
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    fig, ax = plt.subplots(figsize=(10, 2.5))
    ax.axis("off")
    table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    plt.title("Summary of Model Performance", fontsize=14)
    pdf.savefig()
    plt.close()

print(f"✅ PDF report saved to: {report_path}")

✅ PDF report saved to: C:/QGIS/diagnostics/model_report.pdf


In [3]:
# -----------------------------------------------------------
# 🤖 Inference: Run YOLOv8 Segmentation on Tiled Map Images
#
# This step uses a trained YOLOv8 segmentation model to:
# - Predict object masks for each 1024×1024 tile image
# - Save polygon .txt files (YOLO format)
# - Optionally save visualized overlays
#
# It processes all tiles from the specified map folder:
# - /[Map Folder]/tiled/images/
# and outputs:
# - Polygon .txt labels to: /[Map Folder]/predictions/predict_txt/labels/
# - Optional .jpg overlays to: /[Map Folder]/predictions/predict_txt/images/
#
# Classes:
#     • 0 = Water
#     • 1 = Road
#     • 2 = PVeg (Perennial Vegetation)
#
# Parameters:
# - imgsz: Image size used during training (e.g. 640)
# - conf: Confidence threshold (e.g. 0.1 for permissive detection)
# - retina_masks: True for high-quality mask rendering
# - save_txt: Enables saving polygon .txt files (for reconstruction)
#
# Output:
# - One .txt polygon label file per tile (normalized coordinates)
# - Used in downstream mask and shapefile generation
#
# Notes:
# - This step does not create mask images directly, but polygon .txt labels
# - Masks are generated in the following step by rasterizing these polygons
# - Use consistent `imgsz` and class mapping with training
# -----------------------------------------------------------

from ultralytics import YOLO
import torch
import os

# --- Config ---
map_folder = "Flight_2_25pct"
base_dir = "C:/QGIS"
image_dir = os.path.join(base_dir, map_folder, "tiled", "images")

output_name = "predict_txt"
output_dir = os.path.join(base_dir, map_folder, "predictions")

# --- Load model ---
model_path = os.path.join(base_dir, "runs", "segment", "train", "main_model9", "weights", "best.pt")
model = YOLO(model_path)

# --- Predict with manageable size ---
model.predict(
    source=image_dir,
    imgsz=640,  # Lower if needed: 640, 768, etc.
    conf=0.01,
    save=False,
    save_txt=True,
    save_conf=False,
    retina_masks=False,
    name=output_name,
    project=output_dir,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print(f"✅ Inference complete. Check: {output_dir}/{output_name}/labels/")


image 1/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_0.png: 640x640 (no detections), 122.0ms
image 2/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_1.png: 640x640 (no detections), 81.3ms
image 3/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_10.png: 640x640 (no detections), 70.2ms
image 4/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_100.png: 640x640 205 PVegs, 74.5ms
image 5/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_101.png: 640x640 250 PVegs, 80.9ms
image 6/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_102.png: 640x640 300 PVegs, 107.7ms
image 7/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_103.png: 640x640 234 PVegs, 67.1ms
image 8/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_104.png: 640x640 80 PVegs, 88.6ms
image 9/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_105.png: 640x640 (no detections), 74.7ms
image 10/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_106.png: 640x640 (no detections), 78.3ms
image 11/252 C:\QGIS\Flight_2_25pct\tiled\images\chip_107.png: 640x640 (no detections), 79.1

In [4]:
# -----------------------------------------------------------
# 🗺️ Final Map Output: Generate Georeferenced Shapefile from Predicted Masks
# -----------------------------------------------------------

import os
import cv2
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from tqdm import tqdm

# --- Config ---
map_folder = "Flight_2_25pct"
base_dir = f"C:/QGIS/{map_folder}"
mask_dir = os.path.join(base_dir, "predictions", "predict_txt", "labels")
tile_metadata_path = os.path.join(base_dir, "tiled", "tile_metadata.csv")
raster_shape_path = os.path.join(base_dir, "tiled", "raster_shape.txt")
tif_path = os.path.join(base_dir, f"{map_folder}.tiff")
shapefile_path = os.path.join(base_dir, f"{map_folder} Segmentation.shp")

chip_size = 1600        # Original chip size used during tiling
inference_size = 640    # Size used during YOLOv8 inference

# --- Load metadata and raster georeferencing ---
tile_meta = pd.read_csv(tile_metadata_path)
with open(raster_shape_path, "r") as f:
    height, width = map(int, f.read().strip().split(","))

with rasterio.open(tif_path) as src:
    transform = src.transform
    crs = src.crs

# --- Collect all polygons from predicted labels ---
features = []

for _, row in tqdm(tile_meta.iterrows(), total=len(tile_meta), desc="Stitching Tiles"):
    fname, x, y = row["filename"], int(row["x"]), int(row["y"])
    label_path = os.path.join(mask_dir, fname.replace(".png", ".txt"))
    if not os.path.exists(label_path):
        continue

    try:
        with open(label_path, "r") as f:
            lines = f.readlines()
    except:
        continue

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 7:
            continue  # Skip degenerate polygons

        cls_id = int(float(parts[0]))
        coords = list(map(float, parts[1:]))

        pts = np.array(coords, dtype=np.float32).reshape(-1, 2)
        pts *= chip_size  # Scale to original chip size (not inference size)

        # Offset to global pixel coordinates
        pts[:, 0] += x
        pts[:, 1] += y

        # Convert pixel to geographic coordinates
        geo_pts = [rasterio.transform.xy(transform, y_, x_, offset='center') for x_, y_ in pts]
        poly = Polygon(geo_pts)

        if poly.is_valid and poly.area > 0:
            features.append({
                "geometry": poly,
                "class_id": cls_id
            })

# --- Export to shapefile ---
gdf = gpd.GeoDataFrame(features, crs=crs)
gdf.to_file(shapefile_path)

print(f"✅ Shapefile saved to: {shapefile_path}")

Stitching Tiles: 100%|███████████████████████████████████████████████████████████████| 252/252 [08:50<00:00,  2.10s/it]


✅ Shapefile saved to: C:/QGIS/Flight_2_25pct\Flight_2_25pct Segmentation.shp
